Medical Chatbot

In [1]:
!python -V

Python 3.9.12


In [2]:
%pwd

'd:\\Projects\\medical_chatbot\\medical_chatbot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'd:\\Projects\\medical_chatbot\\medical_chatbot'

In [5]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
# extract data from PDF file

def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    
    documents = loader.load()
    return documents

In [7]:
extracted_data=load_pdf_file(data="data/")

In [8]:
# extracted_data

In [9]:
extracted_data[0]

Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'data\\Medical_book.pdf', 'total_pages': 637, 'page': 0, 'page_label': '1'}, page_content='')

In [10]:
# split the data to chunks

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = text_splitter.split_documents(extracted_data)
    return chunks 

In [11]:
chunks = text_split(extracted_data)

In [12]:
print(f"the number of chunks are {len(chunks)}")


the number of chunks are 6600


In [13]:
# !pip install sentence-transformers langchain_huggingface


In [4]:
# !pip install sentence-transformers

In [15]:
# initialize the embedding_model

from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"      # update senetence-transformers lib if get any error

def get_embedding_model():
    embedding_model = HuggingFaceEmbeddings(
        model_name = model_name  
    )
    return embedding_model
    
    
model = get_embedding_model()



In [30]:
# example query embedding

embedded_query = model.embed_query("how are ya!")
print("embedded_query : ", embedded_query)
print("len of vector emebedding : ", len(embedded_query))

embedded_query :  [-0.05779977887868881, -0.0002557742118369788, 0.09759460389614105, -0.0016256258822977543, -0.03991929069161415, -0.07785193622112274, 0.019434232264757156, -0.04283379018306732, -0.11128520965576172, 0.06736092269420624, -0.03127770498394966, 0.016730675473809242, 0.008982236497104168, -0.0047491081058979034, -0.0008051020558923483, -0.004265693016350269, 0.06613454222679138, -0.0539945587515831, -0.06740239262580872, 0.055374857038259506, -0.02644244395196438, 0.022988200187683105, 0.0068043251521885395, 0.097632996737957, -0.009694925509393215, 0.016316009685397148, -0.056544382125139236, 0.00022365109180100262, 0.032435983419418335, -0.08002381771802902, -0.12439987808465958, 0.07289614528417587, -0.08820057660341263, 0.012791899032890797, -0.013056784868240356, -0.0032957522198557854, 0.006380824837833643, -0.10349643230438232, 0.0027178525924682617, 0.02777894213795662, -0.004367252346128225, 0.002557766390964389, 0.011744244024157524, 0.007527731359004974, 0.0

In [31]:
!pip install pinecone

In [24]:
# Pinecone initialization

import os
from dotenv import load_dotenv
from pinecone import Pinecone

load_dotenv()

# from pinecone import (
#     Pinecone
#     ServerlessSpec,
#     CloudProvider,
#     AwsRegion,
#     VectorType
# )

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))


# # 2. Create an index

# index_config = pc.create_index(
#     name="medical-chatbot",
#     dimension=1536,
#     spec=ServerlessSpec(
#         cloud=CloudProvider.AWS,
#         region=AwsRegion.US_EAST_1
#     ),
#     vector_type=VectorType.DENSE
# )

In [16]:
# Initialize vector store and store embeddings in pinecone

from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore.from_documents(
    embedding=get_embedding_model(),
    index_name="medical-chatbot",
    documents=chunks)

vector_store

NameError: name 'chunks' is not defined

In [25]:
# load existing index

import os
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore.from_existing_index(
    index_name="medical-chatbot",
    embedding=model, # get_embedding_model()
)

vector_store

In [26]:
# setting up retriever

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k" : 3})
retriever


VectorStoreRetriever(tags=['PineconeVectorStore', 'HuggingFaceEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x000001E31E818DF0>, search_kwargs={'k': 3})

In [27]:
retrieved_docs = retriever.invoke("What is Psoriasis")
retrieved_docs

[Document(id='5dd4afb8-3cea-4dc9-bf70-02d8a007fec0', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 248.0, 'page_label': '249', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='Psoriasis —A skin disease in which people have\nitchy, scaly, red patches on the skin.\nSystemic lupus erythematosus (SLE) —A chronic\ndisease that affects the skin, joints, and certain\ninternal organs.\nTourette syndrome—A condition in which a person\nhas tics and other involuntary behavior, such as\nbarking, sniffing, swearing, grunting, and making\nuncontrollable movements.\nTremor—Shakiness or trembling.\nnant while taking this medicine should let their physi-\ncians know right away.'),
 Document(id='d0a3ad24-fe78-47fa-9be9-e26d7df55c3e', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 301.0, 'pa

In [48]:
# %pip install -U langchain-ollama

In [45]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model="llama3.1:latest")
# llm.invoke("how are you")

In [59]:
from langchain_ollama.chat_models import ChatOllama

llm = ChatOllama(model="llama2:latest")

response = llm.invoke("how are you")
print(response.content)


ResponseError: model "llama2:latest" not found, try pulling it first (status code: 404)

In [47]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
    You're an assistant for question-answering task. Use the following piece of retriewed context to answer
    the question. If the answer in not found in the context, please say "I don't know the answer. Use maximum
    three sentence to answer the question to keep it concise.
    {context}
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]   
)



In [48]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [60]:
!ollama run llama3.1:latest "Hello"


^C


In [49]:
response = rag_chain.invoke({"input" : "What is acne?"})

ResponseError: model "llama3.1:latest" not found, try pulling it first (status code: 404)

In [42]:
!ollama list

NAME                     ID              SIZE      MODIFIED      
qwen2.5-coder:1.5b       6d3abb8d2d53    986 MB    3 days ago       
deepseek-r1:7b           0a8c26691023    4.7 GB    3 days ago       
deepseek-r1:1.5b         a42b25d8c10a    1.1 GB    5 weeks ago      
x/llama3.2-vision:11b    25e973636a29    7.9 GB    6 weeks ago      
llama3.1:latest          46e0c10c039e    4.9 GB    6 weeks ago      
llama2:latest            78e26419b446    3.8 GB    12 months ago    
